In [ ]:
import chipwhisperer as cw
scope = cw.scope(prog_speed=10e6)

In [ ]:
#APP = 'p256'
APP = 'p384'

#TARGET_FREQ = 25e6
#TARGET_FREQ = 22e6
TARGET_FREQ = 100e6

STREAM = False # TARGET_FREQ must be max 25 MHz to use streaming

if STREAM:
    assert TARGET_FREQ <= 25e6

In [ ]:
from chipwhisperer.capture.targets.CW310 import CW310
target = cw.target(scope, CW310, bsfile=None)

In [ ]:
target.pll.pll_enable_set(True)
target.pll.pll_outenable_set(False, 0)
target.pll.pll_outenable_set(True, 1)
target.pll.pll_outenable_set(False, 2)
target.pll.pll_outfreq_set(TARGET_FREQ, 1)

In [ ]:
sstarget = cw.target(scope)
sstarget.baud = int(115200 * TARGET_FREQ/100e6)

In [ ]:
skip_program = True
if not target.fpga.isFPGAProgrammed() or not skip_program:
    #bsfile = '../../../../Downloads/lowrisc_systems_chip_earlgrey_cw310_0.1.bit'
    bsfile = '/home/jpnewae/git/opentitan/opentitan_sca_ecc/build/lowrisc_systems_chip_earlgrey_cw310_0.1/synth-vivado/lowrisc_systems_chip_earlgrey_cw310_0.1.bit'
    status = target.fpga.FPGAProgram(open(bsfile, "rb"), exceptOnDoneFailure=False, prog_speed=10e6)
    assert status, "FPGA Done pin failed to go high"

In [ ]:
import sys
# this is specific to my local installation, adjust as needed:
sys.path.append("../../..")
from opentitan.otbn_ecc_hacky.util.fpga.cw_spiflash import *

In [ ]:
#firmware = '../../../../Downloads/otbn_ecdsa_p256_test_fpga_cw310.bin' # original Oct/29 work done on this 
if APP == 'p256':
    firmware = '../../opentitan_sca_ecc/build-out/sw/device/sca/ecc_serial_fpga_cw310.bin'
else:
    firmware = '../../opentitan_sca_ecc/build-out/sw/device/sca/ecc384_serial_fpga_cw310.bin'
prog = SPIProgrammer(firmware, "CW310")

In [ ]:
# keep default setup:
scope.io.tio1 = 'serial_tx'
scope.io.tio2 = 'serial_rx'

In [ ]:
if scope._is_husky:
    scope.clock.clkgen_src = 'extclk'
    scope.clock.clkgen_freq = TARGET_FREQ
    scope.clock.adc_mul = 1
    # we'll be playing with the clock so we don't want to get errors when Husky notices the clock changing:
    scope.clock.extclk_monitor_enabled = False
    scope.gain.db = 13
else:
    scope.clock.adc_src = "extclk_x1"
    scope.gain.db = 13 # TODO- this may not be the best value

In [ ]:
scope.clock

In [ ]:
# these are specific to Husky!
scope.adc.offset = 0
if STREAM:
    scope.adc.stream_mode = True
    scope.adc.bits_per_sample = 8
    if APP == 'p256':
        scope.adc.samples = 8000000
    else:
        scope.adc.samples = 16000000
    scope.adc.decimate = 1
    scope.adc._timeout = 5
    scope.gain.db = 22
else:
    scope.adc.stream_mode = False
    scope.adc.bits_per_sample = 12
    scope.adc.samples = 131070
    scope.adc.decimate = 1
    scope.adc._timeout = 5
    scope.gain.db = 13

In [ ]:
def num_to_bytearray(num, bit_length=256):
    return bytearray([num >> (8*i) & 0xFF for i in range((bit_length - 1) // 8,-1,-1)][::-1])

In [ ]:
# initial FW load:
if TARGET_FREQ != 100e6:
    target.pll.pll_outfreq_set(100e6, 1)
prog.run(target)
if TARGET_FREQ != 100e6:
    target.pll.pll_outfreq_set(TARGET_FREQ, 1)


### Capture loop for streaming captures:

In [ ]:
N = 1
reset_firmware = True
all_cycles = []
waves = []
from tqdm.notebook import trange

for i in trange(N):
    if reset_firmware:
        if TARGET_FREQ != 100e6:
            target.pll.pll_outfreq_set(100e6, 1)
        prog.run(target)
        if TARGET_FREQ != 100e6:
            target.pll.pll_outfreq_set(TARGET_FREQ, 1)
        time.sleep(0.3)
        # needed for CW-lite:
        #scope.clock.reset_adc()
        #assert (scope.clock.adc_locked), "ADC failed to lock"
        sstarget.simpleserial_write('v', bytearray())      
    
    sstarget.simpleserial_write('v', bytearray())
    time.sleep(1)
    scope.arm()
    if scope._is_husky:
        start_cycles = 0
    else:
        start_cycles = scope.adc.trig_count
    #k = num_to_bytearray(0xffffffffffffffffffffffffffffffff00000000000000000000000000000000)
    if i == 0:
        k = num_to_bytearray(0xffffffffffffffffffffffffffffffff0000000000000000000000000000000000000000000000000000000000000000, 384)
    elif i == 1:
        k = num_to_bytearray(0x1fffffffffffffffffffffffffffffff0000000000000000000000000000000000000000000000000000000000000000, 384)
    elif i == 2:
        k = num_to_bytearray(0xffff0000ffff0000ffff0000ffff0000ffff0000ffff0000ffff0000ffff0000ffff0000ffff0000ffff0000ffff0000, 384)
    elif i == 3:
        k = num_to_bytearray(0xff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00, 384)

    sstarget.simpleserial_write('p', k)
    
    ret = scope.capture(poll_done = True)
    # If getting inconsistent results (e.g. variable number of cycles), adding a sufficient sleep below here appears to fix things
    time.sleep(1)
    if ret:
        print('Warning: timeout during capture')

    cycles = scope.adc.trig_count - start_cycles
    all_cycles.append(cycles)
    if cycles != 7534831:
        print("Observed odd number of cycles: %d" % cycles)
    waves.append(scope.get_last_trace())


### Capture loop for non-streamed segmented capture (capturing a single full operation in multiple segments using scope.adc.offset):

In [ ]:
from tqdm.notebook import trange
import numpy as np

reset_firmware = True
if APP == 'p256':
    CYCLES = 9000000
else:
    CYCLES = 24000000
SAMPLES = 131070
scope.adc.samples = SAMPLES
segments = CYCLES // SAMPLES
segments = 1
wave = np.array([])
all_cycles = []

for i in trange(segments):
    if reset_firmware:
        if TARGET_FREQ != 100e6:
            target.pll.pll_outfreq_set(100e6, 1)
        prog.run(target)
        if TARGET_FREQ != 100e6:
            target.pll.pll_outfreq_set(TARGET_FREQ, 1)
        time.sleep(0.3)
        # needed for CW-lite:
        #scope.clock.reset_adc()
        #assert (scope.clock.adc_locked), "ADC failed to lock"
        sstarget.simpleserial_write('v', bytearray())      
    
    sstarget.simpleserial_write('v', bytearray())
    scope.adc.offset = i*SAMPLES
    scope.arm()
    if scope._is_husky:
        start_cycles = 0
    else:
        start_cycles = scope.adc.trig_count

    #k = num_to_bytearray(0xffffffffffffffffffffffffffffffff00000000000000000000000000000000)
    #k = num_to_bytearray(0xffffffffffffffffffffffffffffffff0000000000000000000000000000000000000000000000000000000000000000, 384)
    #k = num_to_bytearray(0xffff0000ffff0000ffff0000ffff0000ffff0000ffff0000ffff0000ffff0000ffff0000ffff0000ffff0000ffff0000, 384)
    k = num_to_bytearray(0xffffffff00000000ffffffff00000000ffffffff00000000ffffffff00000000ffffffff00000000ffffffff00000000, 384)

    sstarget.simpleserial_write('p', k)
    
    ret = scope.capture(poll_done = True)
    # If getting inconsistent results (e.g. variable number of cycles), adding a sufficient sleep below here appears to fix things
    time.sleep(1)
    if ret:
        print('Warning: timeout during capture')

    cycles = scope.adc.trig_count - start_cycles
    all_cycles.append(cycles)
    #if cycles != 7534831:
    #    print("Observed odd number of cycles: %d" % cycles)
        
    wave = np.append(wave, scope.get_last_trace())


In [ ]:
max(all_cycles), min(all_cycles), max(all_cycles) - min(all_cycles)

In [ ]:
max(abs(wave))

## Save results for analysis with otbn_find_bits.ipynb or otbn_attack*.ipynb:

In [ ]:
import numpy as np
#np.save('waves_p384_22M_half1half0.npy', waves)
#np.save('waves_p384_22M_half1half0_32b_patterned.npy', waves)
np.save('waves_p384_22M_set.npy', waves)
#np.save('waves_p384_100M_32b_patterned.npy', wave)
#np.save('waves_p384_100M_160ones96zeros.npy', wave)
#np.save('waves_p384_100M_128ones128zeros128ones.npy', wave)
#np.save('waves_p384_100M_130k_samples.npy', waves)
#np.save('waves_p384_100M_half1half0_again.npy', wave)
#np.save('waves_p384_100M_64b_patterned_again.npy', wave)

### random plotting follows....

In [ ]:
p = figure(plot_width=2000)

#samples = 300000
base = 0
#samples = len(waves[0])
#samples = len(wave)
samples = 200000
xrange = range(samples)
#p.line(xrange, waves[3][base:base+samples], line_color="red")
#p.line(xrange, waves[0][base:base+samples], line_color="green")
#p.line(xrange, waves[0][base:base+samples], line_color="blue")
#p.line(xrange, waves[1][base:base+samples], line_color="orange")
#p.line(xrange, waves[2][base:base+samples], line_color="green")

p.line(xrange, wave[base:base+samples], line_color="blue")

In [ ]:
show(p)

In [ ]:
d = figure(plot_width=2000)

samples = 300000
base = 0
samples = len(waves[0])
xrange = range(samples)
d.line(xrange, waves[0][base:base+samples] - waves[3][base:base+samples], line_color="red")
#d.line(xrange, waves[3][base:base+samples] - waves[2][base:base+samples], line_color="blue")

In [ ]:
show(d)

In [ ]:
import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade
hv.extension('bokeh')
datashade(hv.Curve(waves[0][base:base+samples]-waves[3][base:base+samples])).opts(width=2000, height=900)

In [ ]:
import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')

from bokeh.models import HoverTool

hover = HoverTool(mode='vline')

w0 = datashade(hv.Curve(waves[0][base:base+samples]), cmap=['green'])
w1 = datashade(hv.Curve(waves[1][base:base+samples]), cmap=['blue'])
w2 = datashade(hv.Curve(waves[2][base:base+samples]), cmap=['purple'])
w3 = datashade(hv.Curve(waves[3][base:base+samples]), cmap=['orange'])

wf = datashade(hv.Curve(wf2[base:base+samples]), cmap=['black'])
#(w0 * w1 * w2 * w3).opts(width=2000, height=900, tools=['hover', 'vline'])
#(w2 * w0).opts(width=2000, height=900, tools=['hover', 'vline'])
(w2 * wf).opts(width=2000, height=900)

In [ ]:
#samples = cycles
#samples = int(3e5)

import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')

from bokeh.models import HoverTool

hover = HoverTool(mode='vline')

d0 = datashade(hv.Curve(wf1[base:base+samples] - wf0[base:base+samples]), cmap=['green'])
d1 = datashade(hv.Curve(wf3[base:base+samples] - wf0[base:base+samples]), cmap=['red'])
(d0 * d1).opts(width=2000, height=900, tools=['hover', 'vline'])
#(d1).opts(width=2000, height=900)

In [ ]:
#samples = cycles
#samples = int(3e5)

import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')

from bokeh.models import HoverTool

hover = HoverTool(mode='vline')

d0 = datashade(hv.Curve(waves[1][base:base+samples] - waves[0][base:base+samples]), cmap=['green'])
d1 = datashade(hv.Curve(waves[3][base:base+samples] - waves[0][base:base+samples]), cmap=['red'])
(d1 * d0).opts(width=2000, height=900, tools=['hover', 'vline'])
#(d1).opts(width=2000, height=900)

In [ ]:
import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
from holoviews import opts
import datashader as ds

hv.extension('bokeh')

lines = {i: hv.Curve(waves[i][base:base+samples]) for i in range(len(waves))}
linespread = dynspread(datashade(hv.NdOverlay(lines, kdims='k'), aggregator=ds.by('k', ds.count())))
linespread.opts(opts.RGB(width=2400, height=1200))


In [ ]:
#offset = 11665

import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')

from bokeh.models import HoverTool

hover = HoverTool(mode='vline')

w25  = datashade(hv.Curve(waves25[0][base:base+samples]), cmap=['green'])
w100 = datashade(hv.Curve(waves100[0][base+offset:base+offset+samples]), cmap=['blue'])
(w25 * w100).opts(width=2000, height=900, tools=['hover', 'vline'])
#(w100).opts(width=2000, height=900, tools=['hover', 'vline'])